## Clean Data

Clean data based on EDA analysis. Those features are going to be removed.
  
- recorded_by is the same for all the samples, need to be removed
- num_private has most missing values (98.7%), need to be removed
- amount_tsh has 2nd most missing values (70%), need to be removed
- scheme_name has 47.418% samples with missing value, need to be removed
- wpt_name and subvillage due to so many unique values

- “lga”,“region”,“basin” ,correlated other region information
- “extraction_type_group”,“extraction_type_class”, correlated with ‘extraction_type’
- “management_group”, correlated with ‘management’
- “payment_type”, correlated with ‘payment’
- “quality_group”, correlated with ‘quality’
- “quantity_group”, correlated with ‘quantity’
- “source_type”,“source_class”, correlated with ‘source’
- “waterpoint_type_group”, correlated with ‘waterpoint_type’

In [1]:
import pandas as pd
import numpy as np
import datetime

## Functions to clean the data

In [2]:
def clean_dat(d):
    ids = d['id']
    d = d.drop(labels=['id'],axis=1)
  
    ## Removing those features ###
    # recorded_by is the same for all the samples, need to be removed
    # num_private has most missing values (98.7%), need to be removed
    # amount_tsh has 2nd most missing values (70%), need to be removed
    # scheme_name has 47.418% samples with missing value, need to be removed
    # wpt_name and subvillage due to so many unique values
    # 
    # “lga”,“region”,“basin” ,correlated other region information
    # “extraction_type_group”,“extraction_type_class”, correlated with ‘extraction_type’
    # “management_group”, correlated with ‘management’
    # “payment_type”, correlated with ‘payment’
    # “quality_group”, correlated with ‘quality’
    # “quantity_group”, correlated with ‘quantity’
    # “source_type”,“source_class”, correlated with ‘source’
    # “waterpoint_type_group”, correlated with ‘waterpoint_type’
    removed_labels = ['recorded_by','num_private','scheme_name','wpt_name','subvillage',
                      'lga','region','basin',
                      'extraction_type_group','extraction_type_class',
                      'management_group',
                      'payment_type',
                      'quality_group',
                      'quantity_group',
                      'source_type','source_class',
                      'waterpoint_type_group']
    d = d.drop(labels=removed_labels,axis=1)
    
    ### Convert 0 into NA
    d.loc[d.funder=="0",'funder'] = np.NAN  
    d.loc[d.installer=="0",'installer']= np.NAN
    d.loc[d.installer=="-",'installer']= np.NAN
    d.loc[d.longitude==0,'longitude'] = np.NAN
    d.loc[d.district_code==0,'district_code'] = np.NAN
    d.loc[d.construction_year==0,'construction_year'] = np.NAN
    
    ### date_recorded into year, month, week and days to 2014-01-01
    d['date_recorded_year'] =  [date.split("-")[0] for date in d.date_recorded]
    d['date_recorded_month'] = [date.split("-")[1] for date in d.date_recorded]
    d['date_recorded_week'] = [datetime.datetime.strptime(date,"%Y-%m-%d").weekday()  for date in d.date_recorded]
    d['date_recorded_offset_days'] = [(datetime.datetime(2014,1,1) - datetime.datetime.strptime(date,"%Y-%m-%d")).days for date in d.date_recorded]
    
    d = d.drop(labels="date_recorded",axis=1)
    
    def reduce_category_levels(x,max_levels=30):
        xx = x.value_counts(sort=True)
        if(len(xx)>max_levels):
            keeps = xx.index[:max_levels]
            x = [s if s in keeps else "Others" for s in x]
        return x    
    ## For those 3 categories, reduce to 30 lelves
    ## funder	1896
    ## ward	2092
    ## installer	2143        
    
    d.loc[:,"funder"] = reduce_category_levels(d.funder)
    d.loc[:,'ward'] = reduce_category_levels(d.ward)
    d.loc[:,'installer']=reduce_category_levels(d.installer)
    
    return d

In [3]:
d_train = pd.read_csv("../data/train.csv")
d_test = pd.read_csv("../data/test.csv")
id_test = d_test.id
n = d_train.shape[0]  ## total rows of training


In [4]:
d = d_train.append(d_test)
d_cleaned = clean_dat(d)

In [5]:
d_train_cleaned = d_cleaned.iloc[:n]
d_test_cleaned = d_cleaned.iloc[n:]

In [6]:
d_train_cleaned.to_csv(path_or_buf="../data/train_clean.csv",index=False)
d_test_cleaned.to_csv(path_or_buf="../data/test_clean.csv",index=False)
id_test.to_frame().to_csv(path_or_buf = "../data/test_id.csv",index=False )
